# Build a Prompt Chaining Agentic Workflow Using Strands Agent

In this lab, we will build a chained prompts workflow using Strands Agent to intelligently process a legal contract, draft an email with concerns in the contract, and finally provide feedback on the email. Chaining prompts, which involves using the output of one prompt as input to another, offers several advantages when working with Large Language Models (LLMs).

## What is Prompt Chaining
Prompt chaining is a technique that involves breaking down a workflow into a series of known steps. Then orchestrates each step (typically involve an LLM invocation) to handle the output generated from the previous step. Within the worklow, there could be one or more conditional steps which determines the the trajectory of the workflow based on the given state.

## Architecture Diagram
<img src="../../imgs/prompt-chaining-architecture.png" width=800>

Let's get started!




In [ ]:
%pip install strands-agents strands-agents-tools -Uq

### Import Required Libraries and Configure Logging

Import the necessary libraries including Strands Agent, JSON for data handling, and configure logging with INFO level to track the workflow progress with timestamps.

In [3]:
from strands import Agent
import json
import logging

# Configure logging
logging.basicConfig(
    level=logging.INFO,
    format='%(asctime)s - %(levelname)s - %(message)s',
    datefmt='%Y-%m-%d %H:%M:%S'
)
logger = logging.getLogger(__name__)

### Configure Model

Set the Amazon Nova model ID that will be used by all agents in our prompt chaining workflow.


In [16]:
import boto3
import os
import time

sts_client = boto3.client('sts')
session = boto3.session.Session()

account_id = sts_client.get_caller_identity()["Account"]
region = session.region_name

model_id = "us.amazon.nova-micro-v1:0"
default_bucket_name = f"labs-bucket-{region}-{account_id}" # replace the bucket name if running outside of the AWS facilitated environment.

### Create Concern Finder Agent

Initialize the first Strands agent in our chain - the Chief Legal Officer agent that reviews contracts for risks, focusing on data privacy, SLAs, and liability caps. This agent outputs findings in structured XML format.


In [ ]:
# Create specialized agents
concern_finder_system_prompt = """You’re our Chief Legal Officer. You are given a contract to review for risks, focusing on data privacy, SLAs, and liability caps.

Output your findings in <risks> XML tags.
"""
concernFinderAgent = Agent(model=model_id, system_prompt=concern_finder_system_prompt, callback_handler=None)

### Create Email Writer Agent

Initialize the second Strands agent in our chain - an expert email writer that takes the legal concerns identified by the previous agent and drafts a professional corporate email to Frost Technologies outlining the concerns and proposing contract changes.


In [ ]:
emailWriterAgent_system_prompt = """You are an expert at writing corporate legal emails. You are given the concerns from a contract in <risk> XML tag. Your task is to draft an email to Frost Technologies outlining the following concerns and proposing changes to a contract.
"""
emailWriterAgent = Agent(model=model_id, system_prompt=emailWriterAgent_system_prompt, callback_handler=None)

### Create Email Reviewer Agent

Initialize the third Strands agent in our chain - a professional email reviewer that analyzes the draft email and provides feedback on tone, clarity, and professionalism to ensure the communication meets corporate standards.


In [ ]:
emailReviewerAgent_system_prompt = """You are a professional email reviewer. You are given an email content in <email> XML tag. Your task is to review an email and provide feedback. 
Give feedback on tone, clarity, and professionalism.
"""
emailReviewerAgent = Agent(model=model_id, system_prompt=emailReviewerAgent_system_prompt, callback_handler=None)

### Create Email Rewriter Agent

Initialize the final Strands agent in our chain - a professional email writer that takes the original email and the reviewer's feedback to produce an improved, polished version of the email that incorporates all suggested improvements.


In [ ]:
emailRewriterAgent_system_prompt = """You are a professional email writer. You are given an original email content in <email> XML tag, and the feedback from a reviewer in <feedback> XML tag. Your task is to incorporate the feedback and rewrite the email from the orignial email.
Put your rewritten email in the <rewritten_email> XML tag.
"""
emailRewriterAgent = Agent(model=model_id, system_prompt=emailRewriterAgent_system_prompt, callback_handler=None)

### S3 Document Retrieval Function

Define a function to retrieve contract documents from Amazon S3. This function handles AWS S3 operations with proper error handling and logging for any issues that might occur during document retrieval.


In [9]:
import boto3
from botocore.exceptions import ClientError

def retrieve_from_s3(bucket_name, contract_name):
    """
    Retrieve a contract document from S3
    
    Args:
        bucket_name (str): Name of the S3 bucket
        contract_name (str): Key/filename of the contract in S3
    
    Returns:
        str: Content of the contract document
    """
    try:
        # Create S3 client
        s3_client = boto3.client('s3')
        
        # Get the object from S3
        response = s3_client.get_object(Bucket=bucket_name, Key=contract_name)
        
        # Read and decode the content
        contract_content = response['Body'].read().decode('utf-8')
        
        return contract_content
        
    except ClientError as e:
        logger.error(f"Error retrieving contract from S3: {e}")
        return None
    except Exception as e:
        logger.error(f"Unexpected error: {e}")
        return None

### Main Prompt Chaining Workflow Function

Define the main workflow function that orchestrates the entire prompt chaining process:
1. Retrieves contract from S3
2. Analyzes contract for legal concerns
3. Drafts an email based on concerns
4. Reviews the email for improvements
5. Rewrites the email incorporating feedback

This demonstrates the core concept of prompt chaining where each agent's output becomes the input for the next agent in the sequence.


In [14]:
def contract_moderation_email_workflow(contract_key_dict):
    try:
        contract_data_dict = json.loads(contract_key_dict)
        if "s3_bucket_name" in contract_data_dict:
            bucket_name = contract_data_dict["s3_bucket_name"]
        else:
            bucket_name = default_bucket_name
        contract_s3_key = contract_data_dict["s3_object_key"]
        contract_data = retrieve_from_s3(bucket_name, contract_s3_key)
        formatted_contract_data = f"<contract>\n{contract_data}\n</contract>"
        concernFinderAgent_response = concernFinderAgent(prompt=formatted_contract_data)
        concernFinderAgent_response_str = concernFinderAgent_response.message['content'][0]['text']
        logger.info("CONTRACT CONCERN FINDER RESPONSE:")
        logger.info(concernFinderAgent_response_str)       

        emailWriterAgent_response = emailWriterAgent(concernFinderAgent_response_str)
        emailWriterAgent_response_str = emailWriterAgent_response.message['content'][0]['text']
        formatted_emailWriterAgent_response_str = f"<email>\n{emailWriterAgent_response_str}\n<email>"
        logger.info("EMAIL WRITER AGENT RESPONSE:")
        logger.info(emailWriterAgent_response_str)

        emailReviewerAgent_response = emailReviewerAgent(formatted_emailWriterAgent_response_str)
        emailReviewerAgent_response_str = emailReviewerAgent_response.message['content'][0]['text']
        logger.info("EMAIL REVIEWER AGENT RESPONSE:")
        logger.info(emailReviewerAgent_response_str)
        formatted_emailReviewerAgent_response_str = f"<email>{emailWriterAgent_response_str}</email>\n<feedback>\n{emailReviewerAgent_response_str}\n</feedback>\n"
        
        emailRewriterAgent_response = emailRewriterAgent(formatted_emailReviewerAgent_response_str)
        emailRewriterAgent_response_str = emailRewriterAgent_response.message['content'][0]['text']
        logger.info("EMAIL REWRIER AGENT RESPONSE:")
        logger.info(emailRewriterAgent_response_str)
        return emailRewriterAgent_response_str
    except Exception as e:
        logger.error(f"Error in contract moderation email workflow: {e}")
    



### Execute Prompt Chaining Workflow

Run the complete prompt chaining workflow with a test contract from S3. This demonstrates the end-to-end process of:
- Starting with a legal contract document
- Chaining through multiple AI agents
- Producing a refined, professional email output

The logging will show each step of the process with timestamps, demonstrating how the output of each agent flows to the next.


In [ ]:
logger.info("Starting contract analysis workflow...")
test_input_data = {
    "s3_object_key" : "data/agents/contract.txt"
}
contract_moderation_email_response = contract_moderation_email_workflow(json.dumps(test_input_data))
logger.info("FINAL CONTRACT MODERATION EMAIL:")
logger.info(contract_moderation_email_response)

## Summary: Prompt Chaining Agentic Workflow

### What We Accomplished

In this lab, we successfully implemented a **prompt chaining agentic workflow** using Strands Agent to process legal contracts and generate professional email communications. This demonstrates how multiple AI agents can work together in sequence, with each agent's output serving as input to the next agent in the chain.

### Key Components Built

**Four Specialized Agents:**
   - **Concern Finder Agent** - Analyzes contracts for risks in data privacy, SLAs, and liability caps
   - **Email Writer Agent** - Drafts professional corporate emails based on legal concerns
   - **Email Reviewer Agent** - Reviews emails for tone, clarity, and professionalism
   - **Email Rewriter Agent** - Incorporates feedback to produce polished final emails

### Workflow Process

The prompt chaining workflow follows this sequence:

```
Legal Contract (S3) → Concern Analysis → Email Draft → Email Review → Final Email
```

1. **Document Retrieval**: Contract is fetched from S3 storage
2. **Risk Analysis**: Legal concerns are identified and structured in XML format
3. **Email Drafting**: Professional email is created based on identified concerns
4. **Quality Review**: Email is analyzed for improvements in tone and clarity
5. **Final Refinement**: Email is rewritten incorporating all feedback

### Key Benefits of Prompt Chaining

- **Specialization**: Each agent focuses on a specific task, leading to higher quality outputs
- **Iterative Improvement**: Multiple rounds of refinement produce superior final results
- **Modularity**: Individual agents can be modified or replaced without affecting the entire workflow
- **Transparency**: Each step is logged and visible, making the process auditable
- **Scalability**: Additional agents can be easily added to extend the workflow

### Technical Highlights

- **Structured Data Flow**: XML tags ensure consistent data formatting between agents
- **Error Resilience**: Comprehensive error handling prevents workflow failures
- **Logging Integration**: INFO-level logging provides detailed workflow visibility
- **AWS Integration**: Seamless S3 document retrieval with proper authentication
- **Model Flexibility**: Uses Amazon Nova models but can be adapted for other LLMs

### Real-World Applications

This pattern can be applied to various business scenarios:
- **Contract Analysis**: Legal document review and risk assessment
- **Content Creation**: Multi-stage content development and refinement
- **Customer Communications**: Automated professional correspondence
- **Document Processing**: Complex document transformation workflows
- **Quality Assurance**: Multi-layered review and improvement processes

This lab demonstrates the power of breaking complex tasks into specialized, chained operations that leverage the strengths of multiple AI agents working in concert.